### 구현해야 할 것

pretrained model 을 제대로 read & MultiquantConv와 SwitchableBN에 맞게 뿌려주는 알고리즘이 필요할 시 load 알고리즘 개발
(다만, num_track issue나 switchableBN 이슈에서 안전하게 동작할 지에 대해선 확실치 않음)


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from easydict import EasyDict as edict

from imagenet_model.quant_mv1_my import Model as MobileNetV1
from imagenet_model.quant_mv1_my_234 import Model as MobileNetV1_234
from imagenet_model.quant_mv1_my import QDepthwiseSeparableConv
from imagenet_model.quant_resnet_my import resnet18_quant, resnet34_quant, resnet50_quant

from imagenet_model.quant_conv import QConv, QConv_Tra_Mulit
from imagenet_model.quant_conv_234 import QConv as QConv_234


In [4]:
args =edict()
args['QWeightFlag'] = True
args['QActFlag'] = True
args['bkwd_scaling_factorW'] = 1.0
args['bkwd_scaling_factorA'] = 1.0
args['groups']=4
args['arch'] = 'resnet18_quant'
mbv1 = MobileNetV1(args)

In [30]:
resnet18 = resnet18_quant(args, pretrained=True)

load pretrained full-precision weights
_IncompatibleKeys(missing_keys=['layer1.0.conv1.bkwd_scaling_factorW', 'layer1.0.conv1.bkwd_scaling_factorA', 'layer1.0.conv1.init', 'layer1.0.conv1.uW.0', 'layer1.0.conv1.uW.1', 'layer1.0.conv1.uW.2', 'layer1.0.conv1.uW.3', 'layer1.0.conv1.lW.0', 'layer1.0.conv1.lW.1', 'layer1.0.conv1.lW.2', 'layer1.0.conv1.lW.3', 'layer1.0.conv1.uA.0', 'layer1.0.conv1.uA.1', 'layer1.0.conv1.uA.2', 'layer1.0.conv1.uA.3', 'layer1.0.conv1.lA.0', 'layer1.0.conv1.lA.1', 'layer1.0.conv1.lA.2', 'layer1.0.conv1.lA.3', 'layer1.0.conv1.output_scale.0', 'layer1.0.conv1.output_scale.1', 'layer1.0.conv1.output_scale.2', 'layer1.0.conv1.output_scale.3', 'layer1.0.conv1.output_scale.4', 'layer1.0.conv1.output_scale.5', 'layer1.0.conv1.output_scale.6', 'layer1.0.bn1.bn.0.weight', 'layer1.0.bn1.bn.0.bias', 'layer1.0.bn1.bn.0.running_mean', 'layer1.0.bn1.bn.0.running_var', 'layer1.0.bn1.bn.1.weight', 'layer1.0.bn1.bn.1.bias', 'layer1.0.bn1.bn.1.running_mean', 'layer1.0.bn1.bn.1.r

In [25]:
resnet18.layer2[1].bn1.bn[6].running_mean.shape

torch.Size([512])

In [3]:
checkpoint = torch.load('./mv1-2468/checkpoint.pth.tar', map_location='cpu')
print(checkpoint['state_dict'].keys())

#mbv1.load_state_dict(checkpoint['state_dict'])
woddp_checkpoint = {}
for key, value in checkpoint['state_dict'].items():
    woddp_checkpoint[key.replace('module.','')] = value

mbv1.load_state_dict(woddp_checkpoint)


odict_keys(['module.head.0.weight', 'module.head.1.weight', 'module.head.1.bias', 'module.head.1.running_mean', 'module.head.1.running_var', 'module.head.1.num_batches_tracked', 'module.stage_0_layer_0.body.0.weight', 'module.stage_0_layer_0.body.0.init', 'module.stage_0_layer_0.body.0.uW.0', 'module.stage_0_layer_0.body.0.uW.1', 'module.stage_0_layer_0.body.0.uW.2', 'module.stage_0_layer_0.body.0.uW.3', 'module.stage_0_layer_0.body.0.lW.0', 'module.stage_0_layer_0.body.0.lW.1', 'module.stage_0_layer_0.body.0.lW.2', 'module.stage_0_layer_0.body.0.lW.3', 'module.stage_0_layer_0.body.0.uA.0', 'module.stage_0_layer_0.body.0.uA.1', 'module.stage_0_layer_0.body.0.uA.2', 'module.stage_0_layer_0.body.0.uA.3', 'module.stage_0_layer_0.body.0.lA.0', 'module.stage_0_layer_0.body.0.lA.1', 'module.stage_0_layer_0.body.0.lA.2', 'module.stage_0_layer_0.body.0.lA.3', 'module.stage_0_layer_0.body.1.bn.0.weight', 'module.stage_0_layer_0.body.1.bn.0.bias', 'module.stage_0_layer_0.body.1.bn.0.running_mean

<All keys matched successfully>

In [4]:
woddp_checkpoint = {}
for key, value in checkpoint['state_dict'].items():
    woddp_checkpoint[key.replace('module.','')] = value

mbv1.load_state_dict(woddp_checkpoint)

mbv1_234 = MobileNetV1_234(args)
checkpoint = torch.load('./mv1-234/checkpoint.pth.tar', map_location='cpu')
print(checkpoint['state_dict'].keys())
woddp_checkpoint = {}
for key, value in checkpoint['state_dict'].items():
    woddp_checkpoint[key.replace('module.','')] = value

mbv1_234.load_state_dict(woddp_checkpoint)

odict_keys(['module.head.0.weight', 'module.head.1.weight', 'module.head.1.bias', 'module.head.1.running_mean', 'module.head.1.running_var', 'module.head.1.num_batches_tracked', 'module.stage_0_layer_0.body.0.weight', 'module.stage_0_layer_0.body.0.init', 'module.stage_0_layer_0.body.0.uW.0', 'module.stage_0_layer_0.body.0.uW.1', 'module.stage_0_layer_0.body.0.uW.2', 'module.stage_0_layer_0.body.0.uW.3', 'module.stage_0_layer_0.body.0.lW.0', 'module.stage_0_layer_0.body.0.lW.1', 'module.stage_0_layer_0.body.0.lW.2', 'module.stage_0_layer_0.body.0.lW.3', 'module.stage_0_layer_0.body.0.uA.0', 'module.stage_0_layer_0.body.0.uA.1', 'module.stage_0_layer_0.body.0.uA.2', 'module.stage_0_layer_0.body.0.uA.3', 'module.stage_0_layer_0.body.0.lA.0', 'module.stage_0_layer_0.body.0.lA.1', 'module.stage_0_layer_0.body.0.lA.2', 'module.stage_0_layer_0.body.0.lA.3', 'module.stage_0_layer_0.body.1.bn.0.weight', 'module.stage_0_layer_0.body.1.bn.0.bias', 'module.stage_0_layer_0.body.1.bn.0.running_mean

<All keys matched successfully>

In [61]:
for n, m in mbv1.named_modules():
    if isinstance(m, QConv):
        print(n)
        print("n's shape : ", m.weight.shape)
        for i in range(len(m.lA)):
            print(f"{2*(i+1)} bit")
            print(f"m.lw[{i}]: ",m.lW[i].data)
            print(f"m.uW[{i}]: ", m.uW[i].data)
            print(f"m.lA[{i}]: ", m.lA[i].data)
            print(f"m.uA[{i}]: ", m.uA[i].data)
            print("---"*10)
        print("==="*10)

stage_0_layer_0.body.3
n's shape :  torch.Size([256, 32, 1, 1])
2 bit
m.lw[0]:  tensor(-0.2133)
m.uW[0]:  tensor(0.2054)
m.lA[0]:  tensor([ 4.1253e-07,  0.0000e+00,  1.7445e-06,  0.0000e+00, -1.7207e-07,
         0.0000e+00, -3.5417e-06])
m.uA[0]:  tensor([12.3741,  0.0000, 12.2505,  0.0000, 13.5394,  0.0000, 13.5436])
------------------------------
4 bit
m.lw[1]:  tensor(-0.2133)
m.uW[1]:  tensor(0.2054)
m.lA[1]:  tensor([ 0.0000e+00,  0.0000e+00, -8.5861e-08,  0.0000e+00, -1.0405e-07,
         0.0000e+00, -9.4837e-07])
m.uA[1]:  tensor([ 0.0000,  0.0000, 12.9830,  0.0000, 14.1862,  0.0000, 13.9646])
------------------------------
6 bit
m.lw[2]:  tensor(-0.2133)
m.uW[2]:  tensor(0.2054)
m.lA[2]:  tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -1.1446e-07,
         0.0000e+00,  9.2112e-06])
m.uA[2]:  tensor([ 0.0000,  0.0000,  0.0000,  0.0000, 14.8628,  0.0000, 14.4084])
------------------------------
8 bit
m.lw[3]:  tensor(-0.2133)
m.uW[3]:  tensor(0.2054)
m.lA[3]:  tenso

In [59]:
for n, m in mbv1_234.named_modules():
    if isinstance(m, QConv_234):
        print(n)
        for i in range(len(m.lA)):
            print(f"{2+(i)} bit")
            print(f"m.lw[{i}]: ",m.lW[i].data)
            print(f"m.uW[{i}]: ", m.uW[i].data)
            print(f"m.lA[{i}]: ", m.lA[i].data)
            print(f"m.uA[{i}]: ", m.uA[i].data)
            print("---"*10)
        print("==="*10)

stage_0_layer_0.body.3
2 bit
m.lw[0]:  tensor(-0.2490)
m.uW[0]:  tensor(0.2349)
m.lA[0]:  tensor([ 1.1920e-08, -2.2094e-09, -3.5430e-09,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00])
m.uA[0]:  tensor([14.3971, 14.4050, 14.8043,  0.0000,  0.0000,  0.0000,  0.0000])
------------------------------
3 bit
m.lw[1]:  tensor(-0.3397)
m.uW[1]:  tensor(0.3239)
m.lA[1]:  tensor([ 0.0000e+00,  2.8444e-08, -1.8341e-10,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00])
m.uA[1]:  tensor([ 0.0000, 14.4169, 14.9727,  0.0000,  0.0000,  0.0000,  0.0000])
------------------------------
4 bit
m.lw[2]:  tensor(0.)
m.uW[2]:  tensor(0.)
m.lA[2]:  tensor([0., 0., 0., 0., 0., 0., 0.])
m.uA[2]:  tensor([0., 0., 0., 0., 0., 0., 0.])
------------------------------
5 bit
m.lw[3]:  tensor(0.)
m.uW[3]:  tensor(0.)
m.lA[3]:  tensor([0., 0., 0., 0., 0., 0., 0.])
m.uA[3]:  tensor([0., 0., 0., 0., 0., 0., 0.])
------------------------------
stage_1_layer_0.body.3
2 bit
m.lw[0]:  tensor(-0.2219)


In [7]:
for n,m in mbv1.named_modules():
    if isinstance(m, QConv) or isinstance(m, QConv_Tra_Mulit):
        print(n)
        print(m.groups)
        print(m.weight.shape)
        print("==="*10)

stage_0_layer_0.body.0
128
torch.Size([128, 1, 3, 3])
stage_0_layer_0.body.3
4
torch.Size([256, 32, 1, 1])
stage_1_layer_0.body.0
256
torch.Size([256, 1, 3, 3])
stage_1_layer_0.body.3
4
torch.Size([512, 64, 1, 1])
stage_1_layer_1.body.0
512
torch.Size([512, 1, 3, 3])
stage_1_layer_1.body.3
4
torch.Size([512, 128, 1, 1])
stage_2_layer_0.body.0
512
torch.Size([512, 1, 3, 3])
stage_2_layer_0.body.3
4
torch.Size([1024, 128, 1, 1])
stage_2_layer_1.body.0
1024
torch.Size([1024, 1, 3, 3])
stage_2_layer_1.body.3
4
torch.Size([1024, 256, 1, 1])
stage_3_layer_0.body.0
1024
torch.Size([1024, 1, 3, 3])
stage_3_layer_0.body.3
4
torch.Size([2048, 256, 1, 1])
stage_3_layer_1.body.0
2048
torch.Size([2048, 1, 3, 3])
stage_3_layer_1.body.3
4
torch.Size([2048, 512, 1, 1])
stage_3_layer_2.body.0
2048
torch.Size([2048, 1, 3, 3])
stage_3_layer_2.body.3
4
torch.Size([2048, 512, 1, 1])
stage_3_layer_3.body.0
2048
torch.Size([2048, 1, 3, 3])
stage_3_layer_3.body.3
4
torch.Size([2048, 512, 1, 1])
stage_3_layer_

In [28]:
for i, (n, m) in enumerate(mbv1.named_modules()):
    if isinstance(m, QDepthwiseSeparableConv):
        if i > 20:
            print(n)
            print(m)
            print("==="*10)    
            break

stage_1_layer_0
QDepthwiseSeparableConv(
  (body): Sequential(
    (0): QConv_Tra_Mulit(
      256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=256, bias=False
      (uW): ParameterList(
          (0): Parameter containing: [torch.float32 of size ]
          (1): Parameter containing: [torch.float32 of size ]
          (2): Parameter containing: [torch.float32 of size ]
          (3): Parameter containing: [torch.float32 of size ]
      )
      (lW): ParameterList(
          (0): Parameter containing: [torch.float32 of size ]
          (1): Parameter containing: [torch.float32 of size ]
          (2): Parameter containing: [torch.float32 of size ]
          (3): Parameter containing: [torch.float32 of size ]
      )
      (uA): ParameterList(
          (0): Parameter containing: [torch.float32 of size 7]
          (1): Parameter containing: [torch.float32 of size 7]
          (2): Parameter containing: [torch.float32 of size 7]
          (3): Parameter containing: [t

In [43]:
i = 2
for n, mm in m.named_modules(): 
    if hasattr(mm, 'weight_bit'):
        mm.weight_bit=i
    if hasattr(mm, 'act_bit'):
        mm.act_bit=i

m(torch.randn(1, 64 * i // 2, 16, 16)).shape

torch.Size([1, 128, 8, 8])

In [57]:
from timm.models import create_model, list_models

print(list_models('*mobilenet*'))
fp_model = create_model('mobilenetv2_100', pretrained=True)

for key in fp_model.state_dict().keys():
    if 'bn' in key:
        if 'init' in key:
            print(key)

['mobilenetv2_035', 'mobilenetv2_050', 'mobilenetv2_075', 'mobilenetv2_100', 'mobilenetv2_110d', 'mobilenetv2_120d', 'mobilenetv2_140', 'mobilenetv3_large_075', 'mobilenetv3_large_100', 'mobilenetv3_rw', 'mobilenetv3_small_050', 'mobilenetv3_small_075', 'mobilenetv3_small_100', 'tf_mobilenetv3_large_075', 'tf_mobilenetv3_large_100', 'tf_mobilenetv3_large_minimal_100', 'tf_mobilenetv3_small_075', 'tf_mobilenetv3_small_100', 'tf_mobilenetv3_small_minimal_100']


In [52]:
m.body[0].state_dict().keys()

odict_keys(['weight', 'init', 'uW.0', 'uW.1', 'uW.2', 'uW.3', 'lW.0', 'lW.1', 'lW.2', 'lW.3', 'uA.0', 'uA.1', 'uA.2', 'uA.3', 'lA.0', 'lA.1', 'lA.2', 'lA.3'])

In [40]:
i = 2
for n, mm in m.named_modules():
    if hasattr(mm, 'weight_bit'):
        mm.weight_bit=i
    if hasattr(mm, 'act_bit'):
        mm.act_bit=i

m(torch.randn(1, 64 * i // 2, 16, 16)).shape

torch.Size([1, 128, 8, 8])

In [8]:
test_conv = QConv(256, 512, kernel_size=3, stride=2, padding=1, bias=False, groups=4, args=args)
test_conv.weight.shape

torch.Size([512, 64, 3, 3])

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np

__all__ = ['QConv', 'QConv1x1', 'QConv3x3', 'SwitchableBatchNorm2d', 'QLinear']


class STE_discretizer(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_in, num_levels):
        x = x_in * (num_levels - 1)
        x = torch.round(x)
        x_out = x / (num_levels - 1)
        return x_out

    @staticmethod
    def backward(ctx, g):
        return g, None


In [10]:
class QConv_Tra_Mulit(nn.Conv2d):
    def  __init__(self, in_channels, out_channels, kernel_size, args, stride=1, padding=0, dilation=1, groups=1,
                 bias=True, basewidth=0, expand_groups=0, oneBit_outchannel=-1, oneBit_inchannel=-1,
                 last_conv=False, first_conv=False):
        super(QConv_Tra_Mulit, self).__init__(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias)

        self.quan_weight = args.QWeightFlag
        self.quan_act = args.QActFlag

        if self.quan_weight:
            self.weight_levels = -1
            self.uW = nn.ParameterList([nn.Parameter(data=torch.tensor(0).float()) for _ in range(4)])
            self.lW = nn.ParameterList([nn.Parameter(data=torch.tensor(0).float()) for _ in range(4)])

        if self.quan_act:
            self.act_levels = -1
            self.uA = nn.ParameterList(
                [nn.Parameter(data=torch.tensor([0, 0, 0, 0, 0, 0, 0]).float()) for _ in range(4)])
            self.lA = nn.ParameterList(
                [nn.Parameter(data=torch.tensor([0, 0, 0, 0, 0, 0, 0]).float()) for _ in range(4)])

        self.weight_bit = 2  # defalut
        self.act_bit = 2  # defalut
        self.register_buffer('init', torch.tensor([0]))
        self.STE_discretizer = STE_discretizer.apply

        self.last_conv = last_conv
        self.first_conv = first_conv
        self.basewidth = basewidth
        self.oneBit_outchannel = oneBit_outchannel
        assert not self.last_conv or not self.first_conv

    def weight_quantization(self, weight, group_index):
        if not self.quan_weight or self.weight_bit == 32:
            return weight

        self.weight_levels = 2 ** self.weight_bit
        weight = (weight - self.lW[group_index]) / (self.uW[group_index] - self.lW[group_index])
        weight = weight.clamp(min=0, max=1)  # [0, 1]
        weight = self.STE_discretizer(weight, self.weight_levels)
        weight = (weight - 0.5) * 2  # [-1, 1]
        return weight

    def act_quantization(self, x, group_index):
        if not self.quan_act or self.act_bit == 32:
            return x

        index = self.act_bit - 2
        self.act_levels = 2 ** self.act_bit
        # self.act_levels = 2 ** 8
        x = (x - self.lA[group_index][index]) / (self.uA[group_index][index] - self.lA[group_index][index])
        x = x.clamp(min=0, max=1)  # [0, 1]
        x = self.STE_discretizer(x, self.act_levels)
        return x

    def select(self, weight, act):
        if self.first_conv:
            if self.weight_bit & 1 == 0:
                if self.weight_bit == 2 or self.weight_bit == 8:
                    act = act[:, :(self.weight_bit // 2) * self.basewidth, :, :]
                    return weight[:act.shape[1], :, :, :], act
                elif self.weight_bit == 4 or self.weight_bit == 6:
                    act = act[:, self.basewidth: self.basewidth + (self.weight_bit // 2) * self.basewidth, :, :]
                    return weight[self.basewidth: self.basewidth + act.shape[1], :, :, :], act
            else:
                if self.weight_bit == 3:
                    act = act[:, :(self.weight_bit // 2) * self.basewidth + self.oneBit_outchannel, :, :]
                    return weight[:act.shape[1], :, :, :], act
                elif self.weight_bit == 5:
                    act = act[:, :(self.weight_bit // 2) * self.basewidth + self.oneBit_outchannel, :, :]
                    return weight[self.basewidth: self.basewidth + act.shape[1]], act
                elif self.weight_bit == 7:
                    act = act[:, :(self.weight_bit // 2) * self.basewidth + self.oneBit_outchannel, :, :]
                    return weight[:act.shape[1], :, :, :], act
        else:
            if self.weight_bit & 1 == 0:
                if self.weight_bit == 2 or self.weight_bit == 8:
                    return weight[:act.shape[1], :, :, :], act
                elif self.weight_bit == 4 or self.weight_bit == 6:
                    return weight[self.basewidth: self.basewidth + act.shape[1], :, :, :], act
            else:
                if self.weight_bit == 3:
                    return weight[:act.shape[1], :, :, :], act
                elif self.weight_bit == 5:
                    return weight[self.basewidth: self.basewidth + act.shape[1]], act
                elif self.weight_bit == 7:
                    return weight[:act.shape[1], :, :, :], act

    def conv(self, FPweight_1, FPact_1):
        Qweight_1 = self.group_weight_quantization(FPweight_1)
        Qact_1 = self.group_activation_quantization(FPact_1)

        output = F.conv2d(Qact_1, Qweight_1, self.bias, self.stride, self.padding, self.dilation,
                          Qweight_1.shape[0])  # Qweight_1.shape[0] -> groups
        return output

    def group_activation_quantization(self, FPact_1):
        if (self.weight_bit & 1) == 0:
            tmp_list = []
            FPact_1 = FPact_1.chunk(self.weight_bit // 2, dim=1)
            for i in range(len(FPact_1)):
                tmp = self.act_quantization(FPact_1[i], i)
                tmp_list.append(tmp)
            Qact_1 = torch.cat(tmp_list, dim=1)
            return Qact_1
        else:
            tmp_list = []
            FPact_1, FPact_2 = FPact_1[:, :(self.weight_bit // 2) * self.basewidth, :, :],\
                               FPact_1[:, (self.weight_bit // 2) * self.basewidth:, :, :]
            FPact_1 = FPact_1.chunk(self.weight_bit // 2, dim=1)
            for i in range(len(FPact_1)):
                tmp = self.act_quantization(FPact_1[i], i)
                tmp_list.append(tmp)
            Qact_1 = torch.cat(tmp_list, dim=1)
            Qact_2 = self.act_quantization(FPact_2, self.weight_bit//2)
            return torch.cat([Qact_1, Qact_2], dim=1)

    def group_weight_quantization(self, FPweight_1):
        if (self.weight_bit & 1) == 0:
            tmp_list = []
            FPweight_1 = FPweight_1.chunk(self.weight_bit // 2, dim=0)
            for i, _ in enumerate(FPweight_1):
                tmp = self.weight_quantization(FPweight_1[i], i)
                tmp_list.append(tmp)
            Qweight_1 = torch.cat(tmp_list, dim=0)
            return Qweight_1
        else:
            tmp_list = []
            FPweight_1, FPweight_2 = FPweight_1[:(self.weight_bit // 2) * self.basewidth],\
                                     FPweight_1[(self.weight_bit // 2) * self.basewidth:]
            FPweight_1 = FPweight_1.chunk(self.weight_bit // 2, dim=0)
            for i in range(len(FPweight_1)):
                tmp = self.weight_quantization(FPweight_1[i], i)
                tmp_list.append(tmp)
            Qweight_1 = torch.cat(tmp_list, dim=0)
            Qweight_2 = self.weight_quantization(FPweight_2, self.weight_bit // 2)
            return torch.cat([Qweight_1, Qweight_2], dim=0)

    def initialize(self, x):
        FPweight, FPact = self.weight.detach(), x.detach()
        FPweight_1, FPact_1 = self.select(FPweight, FPact)

        index = self.act_bit - 2

        if self.quan_weight:
            if (self.weight_bit & 1) == 0:
                FPweight_1 = FPweight_1.chunk(self.weight_bit // 2, dim=0)
                for i in range(len(FPweight_1)):
                    self.uW[i].data.fill_(FPweight_1[i].std() * 3.0)
                    self.lW[i].data.fill_(-FPweight_1[i].std() * 3.0)
            else:
                FPweight_1, FPweight_2 = FPweight_1[:(self.weight_bit // 2) * self.basewidth], \
                                         FPweight_1[(self.weight_bit // 2) * self.basewidth:]

                FPweight_1 = FPweight_1.chunk(self.weight_bit // 2, dim=0)
                for i in range(len(FPweight_1)):
                    self.uW[i].data.fill_(FPweight_1[i].std() * 3.0)
                    self.lW[i].data.fill_(-FPweight_1[i].std() * 3.0)

                self.uW[self.weight_bit // 2].data.fill_(FPweight_2.std() * 3.0)
                self.lW[self.weight_bit // 2].data.fill_(-FPweight_2.std() * 3.0)

        if self.quan_act:
            if (self.weight_bit & 1) == 0:
                FPact_1 = FPact_1.chunk(self.weight_bit // 2, dim=1)
                for i in range(len(FPact_1)):
                    self.uA[i][index].data.fill_(FPact_1[i].std() / math.sqrt(1 - 2 / math.pi) * 3.0)
                    self.lA[i][index].data.fill_(FPact_1[i].min())

            else:
                FPact_1, FPact_2 = FPact_1[:, :(self.weight_bit // 2) * self.basewidth, :, :], \
                                   FPact_1[:, (self.weight_bit // 2) * self.basewidth:, :, :]

                FPact_1 = FPact_1.chunk(self.weight_bit // 2, dim=1)
                for i in range(len(FPact_1)):
                    self.uA[i][index].data.fill_(FPact_1[i].std() / math.sqrt(1 - 2 / math.pi) * 3.0)
                    self.lA[i][index].data.fill_(FPact_1[i].min())

                self.uA[self.weight_bit // 2][index].data.fill_(FPact_2.std() / math.sqrt(1 - 2 / math.pi) * 3.0)
                self.lA[self.weight_bit // 2][index].data.fill_(FPact_2.min())

    def forward(self, x):
        if self.init == 1:
            self.initialize(x)

        FPweight, FPact = self.weight, x
        FPweight_1, FPact_1 = self.select(FPweight, FPact)

        output = self.conv(FPweight_1, FPact_1)

        return output


In [11]:
class QConv(nn.Conv2d):
    # ref. https://github.com/ricky40403/DSQ/blob/master/DSQConv.py#L18
    def __init__(self, in_channels, out_channels, kernel_size, args, stride=1, padding=0, dilation=1, groups=1,
                 bias=True, oneBit_outchannel=-1, oneBit_inchannel=-1, last_conv=False, first_conv=False):
        super(QConv, self).__init__(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias)
        self.quan_weight = args.QWeightFlag
        self.quan_act = args.QActFlag
        self.STE_discretizer = STE_discretizer.apply
        self.weight_bit = 2  # defalut
        self.act_bit = 2  # defalut
        self.oneBit_outchannel = oneBit_outchannel
        self.oneBit_inchannel = oneBit_inchannel
        self.last_conv = last_conv
        self.first_conv = first_conv

        if self.quan_weight:
            self.weight_levels = 2 ** 2
            self.uW = nn.ParameterList([nn.Parameter(data=torch.tensor(0).float()) for _ in range(4)])
            self.lW = nn.ParameterList([nn.Parameter(data=torch.tensor(0).float()) for _ in range(4)])
            # not used
            self.register_buffer('bkwd_scaling_factorW', torch.tensor(args.bkwd_scaling_factorW).float())

        if self.quan_act:
            self.act_levels = -1
            self.uA = nn.ParameterList([nn.Parameter(data=torch.tensor([0, 0, 0, 0, 0, 0, 0]).float()) for _ in range(4)])
            self.lA = nn.ParameterList([nn.Parameter(data=torch.tensor([0, 0, 0, 0, 0, 0, 0]).float()) for _ in range(4)])
            # not used
            self.register_buffer('bkwd_scaling_factorA', torch.tensor(args.bkwd_scaling_factorA).float())

        self.register_buffer('init', torch.tensor([0]))

        # not used
        self.output_scale = nn.ParameterList([nn.Parameter(data=torch.tensor(1).float()) for _ in range(7)])
        self.hook_Qvalues = False
        self.buff_weight = None
        self.buff_act = None
        assert not self.last_conv or not self.first_conv

    def weight_quantization(self, weight, group_index):
        if not self.quan_weight or self.weight_bit == 32:
            return weight

        weight = (weight - self.lW[group_index]) / (self.uW[group_index] - self.lW[group_index])
        weight = weight.clamp(min=0, max=1)  # [0, 1]
        weight = self.STE_discretizer(weight, self.weight_levels)
        weight = (weight - 0.5) * 2  # [-1, 1]
        return weight

    def act_quantization(self, x, group_index):
        if not self.quan_act or self.act_bit == 32:
            return x

        index = self.act_bit - 2
        self.act_levels = 2 ** self.act_bit
        x = (x - self.lA[group_index][index]) / (self.uA[group_index][index] - self.lA[group_index][index])
        x = x.clamp(min=0, max=1)  # [0, 1]
        x = self.STE_discretizer(x, self.act_levels)
        return x

    def group_activation_quantization(self, FPact_1, FPact_2=None):
        if (self.weight_bit & 1) == 0:
            tmp_list = []
            FPact_1 = FPact_1.chunk(self.weight_bit // 2, dim=1)
            for i in range(len(FPact_1)):
                tmp = self.act_quantization(FPact_1[i], i)
                tmp_list.append(tmp)
            Qact_1 = torch.cat(tmp_list, dim=1)
            return Qact_1
        else:
            tmp_list = []
            FPact_1 = FPact_1.chunk(self.weight_bit // 2, dim=1)
            for i in range(len(FPact_1)):
                tmp = self.act_quantization(FPact_1[i], i)
                tmp_list.append(tmp)
            Qact_1 = torch.cat(tmp_list, dim=1)
            Qact_2 = self.act_quantization(FPact_2, self.weight_bit//2)
            return Qact_1, Qact_2

    def group_weight_quantization(self, FPweight_1, FPweight_2=None):
        if (self.weight_bit & 1) == 0:
            tmp_list = []
            FPweight_1 = FPweight_1.chunk(self.weight_bit // 2, dim=0)
            for i in range(len(FPweight_1)):
                tmp = self.weight_quantization(FPweight_1[i], i)
                tmp_list.append(tmp)
            Qweight_1 = torch.cat(tmp_list, dim=0)
            return Qweight_1
        else:
            tmp_list = []
            FPweight_1 = FPweight_1.chunk(self.weight_bit // 2, dim=0)
            for i in range(len(FPweight_1)):
                tmp = self.weight_quantization(FPweight_1[i], i)
                tmp_list.append(tmp)
            Qweight_1 = torch.cat(tmp_list, dim=0)
            Qweight_2 = self.weight_quantization(FPweight_2, self.weight_bit // 2)
            return Qweight_1, Qweight_2


    def initialize(self, x):
        FPweight, FPact = self.weight.detach(), x.detach()
        FPweight_1, FPweight_2, FPact_1, FPact_2 = self.select(FPweight, FPact)

        index = self.act_bit - 2

        if self.quan_weight:
            if (self.weight_bit & 1) == 0:
                FPweight_1 = FPweight_1.chunk(self.weight_bit // 2, dim=0)
                for i in range(len(FPweight_1)):
                    self.uW[i].data.fill_(FPweight[i].std() * 3.0)
                    self.lW[i].data.fill_(-FPweight[i].std() * 3.0)

            else:
                FPweight_1 = FPweight_1.chunk(self.weight_bit // 2, dim=0)
                for i in range(len(FPweight_1)):
                    self.uW[i].data.fill_(FPweight[i].std() * 3.0)
                    self.lW[i].data.fill_(-FPweight[i].std() * 3.0)

                self.uW[self.weight_bit // 2].data.fill_(FPweight_2.std() * 3.0)
                self.lW[self.weight_bit // 2].data.fill_(-FPweight_2.std() * 3.0)

        if self.quan_act:
            if (self.weight_bit & 1) == 0:
                FPact_1 = FPact_1.chunk(self.weight_bit // 2, dim=1)
                for i in range(len(FPact_1)):
                    self.uA[i][index].data.fill_(FPact_1[i].std() / math.sqrt(1 - 2 / math.pi) * 3.0)
                    self.lA[i][index].data.fill_(FPact_1[i].min())

            else:
                FPact_1 = FPact_1.chunk(self.weight_bit // 2, dim=1)
                for i in range(len(FPact_1)):
                    self.uA[i][index].data.fill_(FPact_1[i].std() / math.sqrt(1 - 2 / math.pi) * 3.0)
                    self.lA[i][index].data.fill_(FPact_1[i].min())

                self.uA[self.weight_bit // 2][index].data.fill_(FPact_2.std() / math.sqrt(1 - 2 / math.pi) * 3.0)
                self.lA[self.weight_bit // 2][index].data.fill_(FPact_2.min())

    def select(self, weight, act):
        if self.first_conv: # only select output channel
            index_o = self.out_channels // self.groups * (self.weight_bit // 2)
            index_i_fir = self.in_channels // self.groups * (self.weight_bit // 2)
            index_i_sec = self.in_channels // self.groups * ((self.weight_bit + 1) // 2)
            single_len = self.out_channels // self.groups

            if self.weight_bit == 2 or self.weight_bit == 8:
                return weight[:index_o, :, :, :], None, act[:, :index_i_fir, :, :], None
            elif self.weight_bit == 4 or self.weight_bit == 6:
                return weight[single_len: single_len + index_o, :, :, :], None, \
                       act[:, single_len: single_len + index_i_fir, :, :], None

            elif self.weight_bit == 3:
                return weight[:index_o, :, :, :], weight[index_o:index_o + self.oneBit_outchannel, :, :, :], \
                       act[:, :index_i_fir, :, :], act[:, index_i_fir:index_i_sec, :, :]
            elif self.weight_bit == 5:
                return weight[single_len: single_len + index_o, :, :, :], \
                       weight[single_len + index_o:single_len + index_o + self.oneBit_outchannel, :, :, :], \
                       act[:, :index_i_fir, :, :], act[:, index_i_fir:index_i_sec, :, :]
            elif self.weight_bit == 7:
                return weight[:index_o, :, :, :], weight[index_o:index_o + self.oneBit_outchannel, :, :, :], \
                       act[:, :index_i_fir, :, :], act[:, index_i_fir:index_i_sec, :, :]

        elif self.last_conv: # only select input channel
            index_o = self.out_channels // self.groups * (self.weight_bit // 2)
            index_i_fir = self.in_channels // self.groups * (self.weight_bit // 2)
            index_i_sec = self.in_channels // self.groups * (self.weight_bit // 2) + self.oneBit_inchannel
            single_len = self.out_channels // self.groups

            if self.weight_bit == 2 or self.weight_bit == 8:
                return weight[:index_o, :index_i_fir, :, :], None, act, None
            elif self.weight_bit == 4 or self.weight_bit == 6:
                return weight[single_len: single_len+index_o, :index_i_fir, :, :], None, act, None

            elif self.weight_bit == 3:
                return weight[:index_o, :, :, :], \
                       weight[index_o:index_o + self.out_channels // self.groups, :self.oneBit_inchannel, :, :], \
                       act[:, :index_i_fir, :, :], act[:, index_i_fir:, :, :],
            elif self.weight_bit == 5:
                return weight[single_len: single_len + index_o, :, :, :], \
                       weight[single_len + index_o: single_len + index_o + self.out_channels // self.groups, :self.oneBit_inchannel, :, :], \
                       act[:, :index_i_fir, :, :], act[:, index_i_fir:, :, :],
            elif self.weight_bit == 7:
                return weight[:index_o, :, :, :], \
                       weight[index_o:index_o + self.out_channels // self.groups, :self.oneBit_inchannel, :, :], \
                       act[:, :index_i_fir, :, :], act[:, index_i_fir:, :, :],

        else:
            index_o = self.out_channels // self.groups * (self.weight_bit // 2)
            index_i_fir = self.in_channels // self.groups * (self.weight_bit // 2)
            index_i_sec = self.in_channels // self.groups * (self.weight_bit // 2) + self.oneBit_inchannel
            # single_len = self.in_channels // self.groups
            single_len = self.out_channels // self.groups

            if self.weight_bit == 2 or self.weight_bit == 8:
                return weight[:index_o, :, :, :], None, act, None
            elif self.weight_bit == 4 or self.weight_bit == 6:
                return weight[single_len: single_len+index_o, :, :, :], None, act, None

            elif self.weight_bit == 3:
                return weight[:index_o, :, :, :], \
                       weight[index_o:index_o + self.oneBit_outchannel, :self.oneBit_inchannel, :, :], \
                       act[:, :index_i_fir, :, :], act[:, index_i_fir:, :, :],
            elif self.weight_bit == 5:
                return weight[single_len: single_len + index_o, :, :, :], \
                       weight[single_len + index_o: single_len + index_o + self.oneBit_outchannel, :self.oneBit_inchannel, :, :], \
                       act[:, :index_i_fir, :, :], act[:, index_i_fir:, :, :],
            elif self.weight_bit == 7:
                return weight[:index_o, :, :, :], \
                       weight[index_o:index_o + self.oneBit_outchannel, :self.oneBit_inchannel, :, :], \
                       act[:, :index_i_fir, :, :], act[:, index_i_fir:, :, :],

    def conv(self, FPweight_1, FPweight_2, FPact_1, FPact_2, quantized=True):
        if self.first_conv:
            if FPweight_2 is None: # 2, 4, 6, 8

                Qweight_1 = self.group_weight_quantization(FPweight_1)
                Qact_1 = self.group_activation_quantization(FPact_1)

                output = F.conv2d(Qact_1, Qweight_1, self.bias, self.stride, self.padding, self.dilation,
                                  self.weight_bit // 2) # self.weight_bit // 2 :-> groups
            else:

                Qweight_1, Qweight_2 = self.group_weight_quantization(FPweight_1, FPweight_2)
                Qact_1, Qact_2 = self.group_activation_quantization(FPact_1, FPact_2)


                output_1 = F.conv2d(Qact_1, Qweight_1, self.bias, self.stride, self.padding, self.dilation,
                                  self.weight_bit // 2) # self.weight_bit // 2 :-> groups
                output_2 = F.conv2d(Qact_2, Qweight_2, self.bias, self.stride, self.padding, self.dilation,
                                    1) # 1 :-> groups
                output = torch.cat((output_1, output_2), dim=1)

        elif self.last_conv:
            if FPweight_2 is None:  # 2, 4, 6, 8

                Qweight_1 = self.group_weight_quantization(FPweight_1)
                Qact_1 = self.group_activation_quantization(FPact_1)


                output = F.conv2d(Qact_1, Qweight_1, self.bias, self.stride, self.padding, self.dilation,
                                  self.weight_bit // 2) # self.weight_bit // 2 :-> groups
            else:

                Qweight_1, Qweight_2 = self.group_weight_quantization(FPweight_1, FPweight_2)
                Qact_1, Qact_2 = self.group_activation_quantization(FPact_1, FPact_2)


                output_1 = F.conv2d(Qact_1, Qweight_1, self.bias, self.stride, self.padding, self.dilation,
                                    self.weight_bit // 2) # self.weight_bit // 2 :-> groups
                output_2 = F.conv2d(Qact_2, Qweight_2, self.bias, self.stride, self.padding, self.dilation,
                                    1) # 1 :-> groups
                output = torch.cat((output_1, output_2), dim=1)

        else:
            if FPweight_2 is None: # 2, 4, 6, 8

                Qweight_1 = self.group_weight_quantization(FPweight_1)
                Qact_1 = self.group_activation_quantization(FPact_1)

                output = F.conv2d(Qact_1, Qweight_1, self.bias, self.stride, self.padding, self.dilation,
                                  self.weight_bit // 2) # self.weight_bit // 2 :-> groups
            else:
                Qweight_1, Qweight_2 = self.group_weight_quantization(FPweight_1, FPweight_2)
                Qact_1, Qact_2 = self.group_activation_quantization(FPact_1, FPact_2)

                output_1 = F.conv2d(Qact_1, Qweight_1, self.bias, self.stride, self.padding, self.dilation,
                                    self.weight_bit // 2)# self.weight_bit // 2 :-> groups
                output_2 = F.conv2d(Qact_2, Qweight_2, self.bias, self.stride, self.padding, self.dilation,
                                    1) # 1 :-> groups
                output = torch.cat((output_1, output_2), dim=1)

        return output

    def forward(self, x):
        if self.init == 1:
            self.initialize(x)

        FPweight, FPact = self.weight, x
        FPweight_1, FPweight_2, FPact_1, FPact_2 = self.select(FPweight, FPact)

        output = self.conv(FPweight_1, FPweight_2, FPact_1, FPact_2, quantized=True)

        return output


In [12]:
g = 4
conv = nn.Conv2d(64, 128, 3, 1, groups=g)
print(conv.weight.shape)

torch.Size([128, 16, 3, 3])
